In [1]:
import os
from datetime import datetime


PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

In [2]:
import data_preparation as dp

#dp.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#dp.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)

In [3]:
# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
train_labels, train_features, train_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length)
test_labels, test_features, test_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length)

train_data = dp.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [4]:
import tensorflow as tf
import gan as gan


g_learning_rate = 0.001
d_real_learning_rate = 0.0001
d_fake_learning_rate = 0.0001
noise_dimensions = 100
batch_size = 200
g_n_outputs = 2
d_n_inputs = 2

tf.reset_default_graph()
tf.set_random_seed(42)

noise = tf.placeholder(tf.float32, [None, max_policy_history_length, noise_dimensions], name='noise')
noise_seq_length = tf.placeholder(tf.int32, [None], name='noise_seq_length')

X = tf.placeholder(tf.float32, [None, max_policy_history_length, d_n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")
X_seq_length = tf.placeholder(tf.int32, [None], name="X_seq_length")

g_data = gan.generator(noise=noise, n_outputs=g_n_outputs, seq_length=noise_seq_length)

logits_d_for_X, y_pred_X = gan.discriminator(X=X, seq_length=X_seq_length, n_outputs=max_policy_history_length)
logits_d_for_g, y_pred_g = gan.discriminator(X=g_data, seq_length=noise_seq_length, n_outputs=max_policy_history_length, reuse=True)

g_loss, accuracy_g = gan.generator_loss(batch_size=batch_size, logits=logits_d_for_g)
d_loss_real, accuracy_d_real = gan.discriminator_loss_real(batch_size=batch_size, logits=logits_d_for_X, y=y)
d_loss_fake, accuracy_d_fake = gan.discriminator_loss_fake(batch_size=batch_size, logits=logits_d_for_g)

g_trainer = gan.generator_trainer(g_learning_rate, g_loss)
d_trainer_real = gan.discriminator_trainer_real(d_real_learning_rate, d_loss_real)
d_trainer_fake = gan.discriminator_trainer_fake(d_fake_learning_rate, d_loss_fake)

tf.summary.scalar('Generator_loss', g_loss)
tf.summary.scalar('Discriminator_loss_real', d_loss_real)
tf.summary.scalar('Discriminator_loss_fake', d_loss_fake)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [6]:
import numpy as np


sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

n_epochs = 20

# Pre-train discriminator
for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // batch_size):
        noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
        noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
    
        y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
    
        _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
            sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
                     {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
                      noise: noise_batch, noise_seq_length: noise_seq_length_batch})

    print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
          "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train)

Epoch: 0 dLossReal: 51.1334 acc_real_train: 0.745 dLossFake: 0.409864 acc_fake_train: 1.0


Epoch: 1 dLossReal: 6.80715 acc_real_train: 0.79 dLossFake: 0.000215351 acc_fake_train: 1.0


Epoch: 2 dLossReal: 4.47854 acc_real_train: 0.795 dLossFake: 0.000124225 acc_fake_train: 1.0


Epoch: 3 dLossReal: 1.88812 acc_real_train: 0.865 dLossFake: 8.85739e-05 acc_fake_train: 1.0


Epoch: 4 dLossReal: 5.08818 acc_real_train: 0.705 dLossFake: 0.000221516 acc_fake_train: 1.0


...

Epoch: 15 dLossReal: 1.6193 acc_real_train: 0.895 dLossFake: 3.9339e-08 acc_fake_train: 1.0


Epoch: 16 dLossReal: 1.16044 acc_real_train: 0.925 dLossFake: 4.35113e-08 acc_fake_train: 1.0


Epoch: 17 dLossReal: 1.13205 acc_real_train: 0.88 dLossFake: 2.02656e-08 acc_fake_train: 1.0


Epoch: 18 dLossReal: 0.873133 acc_real_train: 0.91 dLossFake: 1.90735e-08 acc_fake_train: 1.0


Epoch: 19 dLossReal: 0.656374 acc_real_train: 0.92 dLossFake: 9.53674e-09 acc_fake_train: 1.0


In [7]:
#Train generator and discriminator together
n_epochs = 200

# Pre-train discriminator
for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // batch_size):
        noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
        noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
        
        y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
    
        # Train discriminator on both real and fake data
        _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
            sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
                     {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
                      noise: noise_batch, noise_seq_length: noise_seq_length_batch})
    
        # Train generator
        noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
    
        _, gLoss, acc_g = sess.run([g_trainer, g_loss, accuracy_g], feed_dict={noise: noise_batch, noise_seq_length: noise_seq_length_batch})

    print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
          "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train,
          "gLoss:", gLoss, "acc_g:", acc_g)

Epoch: 0 dLossReal: 289.359 acc_real_train: 0.52 dLossFake: 0.0087164 acc_fake_train: 1.0 gLoss: 6.62249 acc_g: 0.0


Epoch: 1 dLossReal: 146.469 acc_real_train: 0.32 dLossFake: 0.00421564 acc_fake_train: 1.0 gLoss: 8.24388 acc_g: 0.0


Epoch: 2 dLossReal: 534.464 acc_real_train: 0.0 dLossFake: 0.00100036 acc_fake_train: 1.0 gLoss: 12.2682 acc_g: 0.0


Epoch: 3 dLossReal: 127.121 acc_real_train: 0.595 dLossFake: 0.273507 acc_fake_train: 1.0 gLoss: 2.5628 acc_g: 0.0


Epoch: 4 dLossReal: 146.882 acc_real_train: 0.835 dLossFake: 0.04842 acc_fake_train: 1.0 gLoss: 5.12982 acc_g: 0.0


...

Epoch: 67 dLossReal: 2078.09 acc_real_train: 0.745 dLossFake: 619.673 acc_fake_train: 0.0 gLoss: 0.0 acc_g: 1.0


Epoch: 68 dLossReal: 2056.71 acc_real_train: 0.81 dLossFake: 2066.78 acc_fake_train: 0.0 gLoss: 0.0 acc_g: 1.0


Epoch: 69 dLossReal: 109121.0 acc_real_train: 0.755 dLossFake: 1.42084e+06 acc_fake_train: 0.0 gLoss: 0.0 acc_g: 1.0


Epoch: 70 dLossReal: 219683.0 acc_real_train: 0.755 dLossFake: 5564.44 acc_fake_train: 0.035 gLoss: 3667.33 acc_g: 0.0


KeyboardInterrupt: 